# Homework 5

## Question 1

uv --version 

uv 0.7.16 (b6b7409d1 2025-06-27)

## Question 2

sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e


## Question 3

The probability that this lead will convert is 0.534.


# Question 4

In [ ]:
import pickle
from fastapi import FastAPI
from pydantic import BaseModel

with open('pipeline_v1.bin', 'rb') as f:
    pipeline = pickle.load(f)

app = FastAPI()

class LeadData(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

@app.get("/")
def read_root():
    return {"message": "Lead Scoring API"}

@app.post("/predict")
def predict_lead_conversion(lead_data: LeadData):
    record = {
        "lead_source": lead_data.lead_source,
        "number_of_courses_viewed": lead_data.number_of_courses_viewed,
        "annual_income": lead_data.annual_income
    }
    
    # Get probability
    probability = pipeline.predict_proba([record])[0][1]
    
    return {"probability": probability}


In [ ]:
import requests

url = "http://localhost:8000/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
result = response.json()

print(f"Probability that this client will get a subscription: {result['probability']:.3f}")


# Question 5

In [ ]:
docker pull agrigorev/zoomcamp-model:2025

docker images agrigorev/zoomcamp-model:2025


The size of the base image `agrigorev/zoomcamp-model:2025` is 181MB (121MB closest match)


## Question 6

Dockerfile:
```dockerfile
FROM agrigorev/zoomcamp-model:2025

# Install FastAPI, uvicorn, requests, and scikit-learn
RUN pip install fastapi uvicorn requests scikit-learn==1.6.1

# Copy the pipeline file and FastAPI script
COPY pipeline_v1.bin main.py ./

# Expose port 8000
EXPOSE 8000

# Run the FastAPI application
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
```


Build and run commands:
```bash
# Build the Docker image
docker build -t lead-scoring-api .

# Run the container
docker run -d -p 8000:8000 --name lead-scoring-container lead-scoring-api

# Test the API
curl -X POST "http://localhost:8000/predict" \
  -H "Content-Type: application/json" \
  -d '{"lead_source": "organic_search", "number_of_courses_viewed": 4, "annual_income": 80304.0}'
```